In [9]:
pip install glob

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement glob (from versions: none)
ERROR: No matching distribution found for glob


In [12]:
import pandas as pd
import glob
import os
import re # Menggunakan re untuk pemrosesan nama file yang lebih baik

# --- Konfigurasi ---
# Ganti dengan path ke folder tempat file-file Anda berada. 
# Jika file berada di folder yang sama dengan script ini, gunakan './'
path_to_files = './' 
file_pattern = '/*.csv' 
output_filename = 'data_gabungan_wilayah.csv'

# Daftar untuk menyimpan setiap DataFrame dari setiap file
all_data = []

# Gunakan glob untuk menemukan semua file CSV
full_pattern = path_to_files + file_pattern
file_list = glob.glob(full_pattern)

print(f"Ditemukan {len(file_list)} file CSV untuk digabungkan.")

# Iterasi melalui setiap file
for filename in file_list:
    try:
        # 1. Ekstrak nama wilayah (kota) dari nama file
        base_name = os.path.basename(filename)
        
        # Menggunakan regex untuk mengambil semua karakter dari awal 
        # hingga sebelum pola tanggal (e.g., ' 2022-01-01')
        match = re.search(r'(.+?)\s\d{4}-\d{2}-\d{2}', base_name)
        
        wilayah = None
        if match:
            # Mengambil group pertama dan mengganti underscore dengan spasi (jika diinginkan)
            wilayah = match.group(1).replace('_', ' ') 
            wilayah_clean = wilayah.strip() # Menghilangkan spasi ekstra di akhir
        else:
            # Fallback jika pola tanggal tidak ditemukan, ambil semua sebelum ekstensi
            wilayah_clean = base_name.replace('.csv', '').split(' 2022-01-01')[0].replace('_', ' ').strip()
        
        if not wilayah_clean:
            print(f"Gagal mengidentifikasi wilayah untuk file: {base_name}. Melewati.")
            continue

        # 2. Baca file CSV
        # Gunakan 'sep=','' jika file menggunakan koma sebagai pemisah, atau ganti jika berbeda.
        df = pd.read_csv(filename, sep=',') 

        # 3. Tambahkan kolom baru 'Wilayah'
        df['Wilayah'] = wilayah_clean.title() # Menggunakan .title() untuk Kapitalisasi yang Rapi

        # 4. Tambahkan DataFrame yang sudah diproses ke dalam list
        all_data.append(df)
        print(f"  Berhasil memproses: {base_name}. Wilayah: {wilayah_clean.title()}")

    except pd.errors.EmptyDataError:
        print(f"  Peringatan: File {filename} kosong. Melewati.")
    except Exception as e:
        print(f"  Gagal memproses file {filename}: {e}")

# --- Proses Penggabungan ---
if all_data:
    # Gabungkan semua DataFrame dalam list menjadi satu DataFrame
    df_combined = pd.concat(all_data, ignore_index=True)

    # 5. Simpan DataFrame hasil gabungan ke file CSV baru
    # encoding='utf-8' memastikan karakter khusus (jika ada) disimpan dengan benar.
    df_combined.to_csv(output_filename, index=False, encoding='utf-8')

    print("-" * 50)
    print(f"✅ Penggabungan Selesai. Data disimpan ke: {output_filename}")
    print(f"Total baris data yang digabungkan: {len(df_combined)}")
    print("-" * 50)
else:
    print("-" * 50)
    print("❌ Tidak ada data yang berhasil diproses atau digabungkan.")
    print("-" * 50)

Ditemukan 9 file CSV untuk digabungkan.
  Berhasil memproses: bundaran_hi 2022-01-01 to 2024-12-31.csv. Wilayah: Bundaran Hi
  Berhasil memproses: data_meteorology.csv. Wilayah: Data Meteorology
  Berhasil memproses: jagakarsa 2022-01-01 to 2024-12-31.csv. Wilayah: Jagakarsa
  Berhasil memproses: jakarta_gbk 2022-01-01 to 2024-12-31.csv. Wilayah: Jakarta Gbk
  Berhasil memproses: kebun_jeruk 2022-01-01 to 2024-12-31.csv. Wilayah: Kebun Jeruk
  Berhasil memproses: kelapa_gading 2022-01-01 to 2024-12-31.csv. Wilayah: Kelapa Gading
  Berhasil memproses: lubang_buaya 2022-01-01 to 2024-12-31.csv. Wilayah: Lubang Buaya
  Berhasil memproses: us_embassy_1 2022-01-01 to 2024-12-31.csv. Wilayah: Us Embassy 1
  Berhasil memproses: us_embassy_2 2022-01-01 to 2024-12-31.csv. Wilayah: Us Embassy 2
--------------------------------------------------
✅ Penggabungan Selesai. Data disimpan ke: data_gabungan_wilayah.csv
Total baris data yang digabungkan: 14248
--------------------------------------------